<a href="https://colab.research.google.com/github/Ashleyyy66/e/blob/main/Chapter1_Data_Colocating_S2_S3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colocating Sentinel-3 OLCI/SRAL and Sentinal-2 Optical Data
In this section, we embark on a detailed exploration of colocating Sentinel-3 data with Sentinel-2 optical data. Colocation of data from these two satellite missions enables a powerful synergy, harnessing the high spatial resolution of Sentinel-2 and the comprehensive coverage and colocated altimeter data from Sentinel-3. This fusion of datasets provides a richer, more detailed perspective of Earth's surface.

In the following sections, we will guide you through the necessary steps to identify and align these datasets.

Week 4 Materials are available [here](https://drive.google.com/drive/folders/1SxmGM9_UJk-M5bEOoTfM_4urvr0257H3?usp=share_link).

## Step 0: Read in Functions Needed

To streamline our data fetching and processing, we'll first load the essential functions. These functions are identical to what we have for the data_fetching notebook in week 3. These functions essentially help you get metadata for the 2 satellites you care about.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datetime import datetime, timedelta
from shapely.geometry import Polygon, Point, shape
import numpy as np
import requests
import pandas as pd
from xml.etree import ElementTree as ET
import os
import json
import folium


def make_api_request(url, method="GET", data=None, headers=None):
    global access_token
    if not headers:
        headers = {"Authorization": f"Bearer {access_token}"}

    response = requests.request(method, url, json=data, headers=headers)
    if response.status_code in [401, 403]:
        global refresh_token
        access_token = refresh_access_token(refresh_token)
        headers["Authorization"] = f"Bearer {access_token}"
        response = requests.request(method, url, json=data, headers=headers)
    return response


def query_sentinel3_olci_arctic_data(start_date, end_date, token):
    """
    Queries Sentinel-3 OLCI data within a specified time range from the Copernicus Data Space,
    targeting data collected over the Arctic region.

    Parameters:
    start_date (str): Start date in 'YYYY-MM-DD' format.
    end_date (str): End date in 'YYYY-MM-DD' format.
    token (str): Access token for authentication.

    Returns:
    DataFrame: Contains details about the Sentinel-3 OLCI images.
    """

    all_data = []
    arctic_polygon = "POLYGON((-180 60, 180 60, 180 90, -180 90, -180 60))"
    # arctic_polygon = (
    #     "POLYGON ((-81.7 71.7, -81.7 73.8, -75.1 73.8, -75.1 71.7, -81.7 71.7))"
    # )

    filter_string = (
        f"Collection/Name eq 'SENTINEL-3' and "
        f"Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/Value eq 'OL_1_EFR___') and "
        f"ContentDate/Start gt {start_date}T00:00:00.000Z and ContentDate/Start lt {end_date}T23:59:59.999Z"
    )

    next_url = (
        f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?"
        f"$filter={filter_string} and "
        f"OData.CSC.Intersects(area=geography'SRID=4326;{arctic_polygon}')&"
        f"$top=1000"
    )

    headers = {"Authorization": f"Bearer {token}"}

    while next_url:
        response = make_api_request(next_url, headers=headers)
        if response.status_code == 200:
            data = response.json()["value"]
            all_data.extend(data)
            next_url = response.json().get("@odata.nextLink")
        else:
            print(f"Error fetching data: {response.status_code} - {response.text}")
            break

    return pd.DataFrame(all_data)


def get_access_and_refresh_token(username, password):
    """Retrieve both access and refresh tokens."""
    url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
    data = {
        "grant_type": "password",
        "username": username,
        "password": password,
        "client_id": "cdse-public",
    }
    response = requests.post(url, data=data)
    response.raise_for_status()
    tokens = response.json()
    return tokens["access_token"], tokens["refresh_token"]


def refresh_access_token(refresh_token):
    """Attempt to refresh the access token using the refresh token."""
    url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
    data = {
        "grant_type": "refresh_token",
        "refresh_token": refresh_token,
        "client_id": "cdse-public",
    }
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    try:
        response = requests.post(url, headers=headers, data=data)
        response.raise_for_status()  # This will throw an error for non-2xx responses
        return response.json()["access_token"]
    except requests.exceptions.HTTPError as e:
        print(f"Failed to refresh token: {e.response.status_code} - {e.response.text}")
        if e.response.status_code == 400:
            print("Refresh token invalid, attempting re-authentication...")
            # Attempt to re-authenticate
            username = username
            password = password
            # This requires securely managing the credentials, which might not be feasible in all contexts
            access_token, new_refresh_token = get_access_and_refresh_token(
                username, password
            )  # This is a placeholder
            refresh_token = (
                new_refresh_token  # Update the global refresh token with the new one
            )
            return access_token
        else:
            raise

def download_single_product(
    product_id, file_name, access_token, download_dir="downloaded_products"
):
    """
    Download a single product from the Copernicus Data Space.

    :param product_id: The unique identifier for the product.
    :param file_name: The name of the file to be downloaded.
    :param access_token: The access token for authorization.
    :param download_dir: The directory where the product will be saved.
    """
    # Ensure the download directory exists
    os.makedirs(download_dir, exist_ok=True)

    # Construct the download URL
    url = (
        f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value"
    )

    # Set up the session and headers
    headers = {"Authorization": f"Bearer {access_token}"}
    session = requests.Session()
    session.headers.update(headers)

    # Perform the request
    response = session.get(url, headers=headers, stream=True)

    # Check if the request was successful
    if response.status_code == 200:
        # Define the path for the output file
        output_file_path = os.path.join(download_dir, file_name + ".zip")

        # Stream the content to a file
        with open(output_file_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
        print(f"Downloaded: {output_file_path}")
    else:
        print(
            f"Failed to download product {product_id}. Status Code: {response.status_code}"
        )

def query_sentinel3_sral_arctic_data(start_date, end_date, token):
    """
    Queries Sentinel-3 SRAL data within a specified time range from the Copernicus Data Space,
    targeting data collected over the Arctic region.

    Parameters:
    start_date (str): Start date in 'YYYY-MM-DD' format.
    end_date (str): End date in 'YYYY-MM-DD' format.
    token (str): Access token for authentication.

    Returns:
    DataFrame: Contains details about the Sentinel-3 SRAL images.
    """

    all_data = []
    # arctic_polygon = "POLYGON((-180 60, 180 60, 180 90, -180 90, -180 60))"
    arctic_polygon = (
        "POLYGON ((-81.7 71.7, -81.7 73.8, -75.1 73.8, -75.1 71.7, -81.7 71.7))"
    )

    filter_string = (
        f"Collection/Name eq 'SENTINEL-3' and "
        f"Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/Value eq 'SR_2_LAN_SI') and "
        f"ContentDate/Start gt {start_date}T00:00:00.000Z and ContentDate/Start lt {end_date}T23:59:59.999Z"
    )

    next_url = (
        f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?"
        f"$filter={filter_string} and "
        f"OData.CSC.Intersects(area=geography'SRID=4326;{arctic_polygon}')&"
        f"$top=1000"
    )

    headers = {"Authorization": f"Bearer {token}"}

    while next_url:
        response = make_api_request(
            next_url, headers={"Authorization": f"Bearer {token}"}
        )
        if response.status_code == 200:
            data = response.json()["value"]
            all_data.extend(data)
            next_url = response.json().get("@odata.nextLink")
        else:
            print(f"Error fetching data: {response.status_code} - {response.text}")
            break

    return pd.DataFrame(all_data)


def query_sentinel2_arctic_data(
    start_date,
    end_date,
    token,
    min_cloud_percentage=10,
    max_cloud_percentage=50,
):
    """
    Queries Sentinel-2 data within a specified time range from the Copernicus Data Space,
    considering a range of cloud coverage by treating greater than and less than conditions as separate attributes.
    Handles pagination to fetch all available data.

    Parameters:
    start_date (str): Start date in 'YYYY-MM-DD' format.
    end_date (str): End date in 'YYYY-MM-DD' format.
    token (str): Access token for authentication.
    min_cloud_percentage (int): Minimum allowed cloud coverage.
    max_cloud_percentage (int): Maximum allowed cloud coverage.

    Returns:
    DataFrame: Contains details about the Sentinel-2 images.
    """

    all_data = []
    arctic_polygon = "POLYGON((-180 60, 180 60, 180 90, -180 90, -180 60))"

    filter_string = (
        f"Collection/Name eq 'SENTINEL-2' and "
        f"Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/Value ge {min_cloud_percentage}) and "
        f"Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/Value le {max_cloud_percentage}) and "
        f"ContentDate/Start gt {start_date}T00:00:00.000Z and ContentDate/Start lt {end_date}T23:59:59.999Z"
    )

    next_url = (
        f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?"
        f"$filter={filter_string} and "
        f"OData.CSC.Intersects(area=geography'SRID=4326;{arctic_polygon}')&"
        f"$top=1000"
    )

    headers = {"Authorization": f"Bearer {token}"}

    while next_url:
        response = make_api_request(
            next_url, headers={"Authorization": f"Bearer {token}"}
        )
        if response.status_code == 200:
            data = response.json()["value"]
            all_data.extend(data)
            next_url = response.json().get("@odata.nextLink")
        else:
            print(f"Error fetching data: {response.status_code} - {response.text}")
            break

    return pd.DataFrame(all_data)


def plot_results(results):
    m = folium.Map(location=[0, 0], zoom_start=2)
    for idx, row in results.iterrows():
        try:
            geojson1 = json.loads(row["Satellite1_Footprint"].replace("'", '"'))
            geojson2 = json.loads(row["Satellite2_Footprint"].replace("'", '"'))

            folium.GeoJson(geojson1, name=row["Satellite1_Name"]).add_to(m)
            folium.GeoJson(geojson2, name=row["Satellite2_Name"]).add_to(m)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

    folium.LayerControl().add_to(m)
    return m


def parse_geofootprint(footprint):
    """
    Parses a JSON-like string to extract the GeoJSON and convert to a Shapely geometry.
    """
    try:
        geo_json = json.loads(footprint.replace("'", '"'))
        return shape(geo_json)
    except json.JSONDecodeError:
        return None


def check_collocation(
    df1, df2, start_date, end_date, time_window=pd.to_timedelta("1 day")
):

    collocated = []
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    for idx1, row1 in df1.iterrows():
        footprint1 = parse_geofootprint(row1["GeoFootprint"])
        if footprint1 is None:
            continue

        s1_start = row1["ContentDate.Start"]
        s1_end = row1["ContentDate.End"]

        if s1_end < start_date or s1_start > end_date:
            continue

        s1_start_adjusted = s1_start - time_window
        s1_end_adjusted = s1_end + time_window

        for idx2, row2 in df2.iterrows():
            footprint2 = parse_geofootprint(row2["GeoFootprint"])
            if footprint2 is None:
                continue

            s2_start = row2["ContentDate.Start"]
            s2_end = row2["ContentDate.End"]

            if s2_end < start_date or s2_start > end_date:
                continue
            if max(s1_start_adjusted, s2_start) <= min(s1_end_adjusted, s2_end):
                if footprint1.intersects(footprint2):
                    collocated.append(
                        {
                            "Satellite1_Name": row1["Name"],
                            "Satellite1_ID": row1["Id"],
                            "Satellite1_Footprint": row1["GeoFootprint"],
                            "Satellite2_Name": row2["Name"],
                            "Satellite2_ID": row2["Id"],
                            "Satellite2_Footprint": row2["GeoFootprint"],
                            "Overlap_Start": max(
                                s1_start_adjusted, s2_start
                            ).isoformat(),
                            "Overlap_End": min(s1_end_adjusted, s2_end).isoformat(),
                        }
                    )

    return pd.DataFrame(collocated)


def make_timezone_naive(dt):
    """Convert a timezone-aware datetime object to timezone-naive in local time."""
    return dt.replace(tzinfo=None)


## Step 1: Get the Metadata for satellites (Sentinel-2 and Sentinel-3 OLCI in this case)
In this example, we illustrate how we co-locate Sentinel-2 and Sentinel-3 OLCI by fetching the metadata first (the same way we did in week 3). Since we are trying to find co-location between 2 satellites, we fetch 2 tables of metadata, representing 2 satellites we care about. In this case, they are named as object
'sentinel3_olci_data' and 'sentinel2_data'.







In [ ]:
username = "zcqsbli@ucl.ac.uk"
password = "lbX041202nb666$$$"
access_token, refresh_token = get_access_and_refresh_token(username, password)
start_date = "2018-06-01"
end_date = "2018-06-02"
path_to_save_data = "/content/drive/MyDrive/Week_4" # Here you can edit where you want to save your metadata
s3_olci_metadata = query_sentinel3_olci_arctic_data(
    start_date, end_date, access_token
)

s2_metadata = query_sentinel2_arctic_data(
    start_date,
    end_date,
    access_token,
    min_cloud_percentage=0,
    max_cloud_percentage=10,
)

# You can also save the metadata
s3_olci_metadata.to_csv(
    path_to_save_data+"sentinel3_olci_metadata.csv",
    index=False,
)

s2_metadata.to_csv(
    path_to_save_data+"sentinel2_metadata.csv",
    index=False,
)

You can try to print them to see what these 2 metadata look like.

In [ ]:
from IPython.display import display

display(s3_olci_metadata)


,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,9db390cf-cfb4-5112-a1ee-8e5f2707401e,S3A_OL_1_EFR____20180601T151119_20180601T15141...,application/octet-stream,665998933,2018-11-01T18:21:34.729000Z,2022-05-27T11:46:15.232535Z,2023-12-07T10:40:04.296086Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-3/OLCI/OL_1_EFR/2018/06/01/S3...,"[{'Value': 'c4196f6241364e344781a6293ee6b664',...","{'Start': '2018-06-01T15:11:19.387000Z', 'End'...",geography'SRID=4326;POLYGON ((-80.6219 84.2705...,"{'type': 'Polygon', 'coordinates': [[[-80.6219..."
1,application/octet-stream,89a02158-6ce3-51f0-a962-578c0cfc3061,S3A_OL_1_EFR____20180602T212905_20180602T21320...,application/octet-stream,675502028,2018-10-28T21:39:59.997000Z,2022-05-27T11:46:17.436495Z,2023-12-07T10:40:55.357408Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-3/OLCI/OL_1_EFR/2018/06/02/S3...,"[{'Value': '37ea08b9364f92107c62a9a8f9eb84bf',...","{'Start': '2018-06-02T21:29:05.362000Z', 'End'...",geography'SRID=4326;POLYGON ((-175.044 84.2832...,"{'type': 'Polygon', 'coordinates': [[[-175.044..."
2,application/octet-stream,58c1eed1-098c-5071-8fa3-568f5b090a53,S3A_OL_1_EFR____20180601T201417_20180601T20171...,application/octet-stream,674085092,2018-11-01T18:23:50.088000Z,2022-05-27T11:46:35.891142Z,2023-12-07T10:43:48.480036Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-3/OLCI/OL_1_EFR/2018/06/01/S3...,"[{'Value': '71a46976fa26198f0a56647a8bbecbcd',...","{'Start': '2018-06-01T20:14:17.050000Z', 'End'...",geography'SRID=4326;POLYGON ((-156.343 84.2811...,"{'type': 'Polygon', 'coordinates': [[[-156.343..."
3,application/octet-stream,0b26e275-2a88-5817-92ef-7ebeeb0033d9,S3A_OL_1_EFR____20180602T180707_20180602T18100...,application/octet-stream,667045878,2018-11-01T18:58:38.315000Z,2022-05-27T11:46:38.757724Z,2023-12-07T10:45:38.755087Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-3/OLCI/OL_1_EFR/2018/06/02/S3...,"[{'Value': '34f90ad8f3325d94df2c778ced86d070',...","{'Start': '2018-06-02T18:07:06.920000Z', 'End'...",geography'SRID=4326;POLYGON ((-124.573 84.2832...,"{'type': 'Polygon', 'coordinates': [[[-124.573..."
4,application/octet-stream,e3c452c7-f8b5-50e9-a9b3-b0bc4b43df50,S3A_OL_1_EFR____20180602T130409_20180602T13070...,application/octet-stream,670413727,2018-11-01T18:56:27.803000Z,2022-05-27T11:46:35.458477Z,2023-12-07T10:44:34.315009Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-3/OLCI/OL_1_EFR/2018/06/02/S3...,"[{'Value': '170011503e3f756ef5285109e04d5dca',...","{'Start': '2018-06-02T13:04:09.258000Z', 'End'...",geography'SRID=4326;POLYGON ((-48.8162 84.2707...,"{'type': 'Polygon', 'coordinates': [[[-48.8162..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,application/octet-stream,16ab2ce3-f5e8-5a02-ab83-ac5cdc858a93,S3B_OL_1_EFR____20180602T230619_20180602T23091...,application/octet-stream,0,2020-05-04T15:04:35.938000Z,2020-05-04T16:12:35.026723Z,2020-05-04T16:12:35.026723Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-3/OLCI/OL_1_EFR/2018/06/02/S3...,[],"{'Start': '2018-06-02T23:06:18.951000Z', 'End'...",geography'SRID=4326;POLYGON ((-15.117827625966...,"{'type': 'Polygon', 'coordinates': [[[-15.1178..."
338,application/octet-stream,2f363e42-e61c-59a4-a51f-f7ccacbc3da4,S3B_OL_1_EFR____20180602T231219_20180602T23151...,application/octet-stream,0,2020-05-04T14:41:59.925000Z,2020-05-04T16:12:41.395265Z,2020-05-04T16:12:41.395265Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-3/OLCI/OL_1_EFR/2018/06/02/S3...,[],"{'Start': '2018-06-02T23:12:18.951000Z', 'End'...",geography'SRID=4326;MULTIPOLYGON (((180 72.052...,"{'type': 'MultiPolygon', 'coordinates': [[[[18..."
339,application/octet-stream,be1ab892-3859-5cf6-a734-6ec0b2eb91e8,S3B_OL_1_EFR____20180602T231519_20180602T23181...,application/octet-stream,0,2020-05-04T15:04:42.445000Z,2020-05-04T16:12:53.766136Z,2020-05-04T16:12:53

In [ ]:
s3_olci_metadata['ContentDate'][0]

{'Start': '2018-06-01T15:11:19.387000Z', 'End': '2018-06-01T15:14:19.387000Z'}

In [ ]:
s2_metadata['ContentDate'][0]

{'Start': '2018-06-01T22:55:29.024000Z', 'End': '2018-06-01T22:55:29.024000Z'}

In [ ]:
from IPython.display import display

display(s2_metadata)
#each row is for the metadata for one image

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,0200999b-0712-4a70-b7a3-4160e5d4f95b,S2B_MSIL1C_20180601T225529_N0500_R001_T03WXU_2...,application/octet-stream,660811240,2024-02-13T01:12:13.265000Z,2024-02-13T02:35:07.552178Z,2024-05-11T15:51:40.346618Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C_N0500/2018/06/01/S2...,"[{'Value': '8c8ac895d615eadb229d86502cecc0b7',...","{'Start': '2018-06-01T22:55:29.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-162.22142 71.18...,"{'type': 'Polygon', 'coordinates': [[[-162.221..."
1,application/octet-stream,e6e03e11-853c-49cb-9558-aa34b3c26f9c,S2A_MSIL1C_20180601T201851_N0500_R071_T16XEP_2...,application/octet-stream,794426484,2024-02-13T07:18:39.550000Z,2024-02-13T09:15:35.205010Z,2024-05-11T16:00:11.788775Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C_N0500/2018/06/01/S2...,"[{'Value': '1ceeaadcc7643fe8274148325823140e',...","{'Start': '2018-06-01T20:18:51.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-81.71936 79.243...,"{'type': 'Polygon', 'coordinates': [[[-81.7193..."
2,application/octet-stream,7d345969-2abf-4de6-a9f4-3320d76c779b,S2A_MSIL2A_20180601T201851_N0500_R071_T16XEN_2...,application/octet-stream,301009271,2024-02-13T09:20:24.635000Z,2024-02-13T11:42:57.376436Z,2024-05-11T16:02:53.861123Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A_N0500/2018/06/01/S2...,"[{'Value': '1f5209d5d064f2f1c55856a639d8e5a9',...","{'Start': '2018-06-01T20:18:51.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-81.936676 79.22...,"{'type': 'Polygon', 'coordinates': [[[-81.9366..."
3,application/octet-stream,9afdca10-4db6-49d5-b7f6-dab854f79873,S2A_MSIL1C_20180601T083651_N0500_R064_T41XNG_2...,application/octet-stream,749681344,2024-03-10T05:35:11.658000Z,2024-03-10T05:44:23.466923Z,2024-05-10T22:38:00.094092Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C_N0500/2018/06/01/S2...,"[{'Value': 'e3e5ee08f91f58c321ee7a3421557c23',...","{'Start': '2018-06-01T08:36:51.024000Z', 'End'...",geography'SRID=4326;POLYGON ((62.9991623279442...,"{'type': 'Polygon', 'coordinates': [[[62.99916..."
4,application/octet-stream,fd916715-6f46-4aa8-8042-2fe26d0ea74b,S2B_MSIL1C_20180601T210819_N0500_R143_T22XDQ_2...,application/octet-stream,33591244,2024-02-10T09:54:37.556000Z,2024-02-11T04:35:12.618487Z,2024-05-09T00:34:15.164461Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C_N0500/2018/06/01/S2...,"[{'Value': 'f6eed87257f571b75c8c2a7aad710f69',...","{'Start': '2018-06-01T21:08:19.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-50.97757 80.324...,"{'type': 'Polygon', 'coordinates': [[[-50.9775..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831,application/octet-stream,5cfe0f0c-6322-47a7-a4c4-d13f061194c1,S2A_MSIL2A_20180602T213531_N0500_R086_T05VNJ_2...,application/octet-stream,1244379894,2024-02-12T06:33:16.350000Z,2024-02-12T07:33:43.245009Z,2024-05-10T22:12:56.246340Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A_N0500/2018/06/02/S2...,"[{'Value': '2809b7aa935a57b0da3e0976ee6223c5',...","{'Start': '2018-06-02T21:35:31.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-153.00038 62.23...,"{'type': 'Polygon', 'coordinates': [[[-153.000..."
1832,application/octet-stream,d1837387-25cd-4157-90b5-4e9e6c446d0f,S2A_MSIL1C_20180602T213531_N0500_R086_T05VNK_2...,application/octet-stream,898493826,2024-02-12T06:03:34.743000Z,2024-02-12T07:22:40.768588Z,2024-05-10T22:12:50.181627Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C_N0500/2018/06/02/S2...,"[{'Value': 'cbd30fb940a42278b56cb0d2f3a6e1a5',...","{'Start': '2018-06-02T21:35:31.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-153.0004 62.927...,"{'type': 'Polygon', 'coordinates': [[[-153.000..."
1833,application/octet-stream,87ab1358-d3ba-4227-a009-07cbd1f8b050,S2A_MSIL2A_20180602T213531_N0500_R0

From above, we can see that there are 342 rows for S3 OLCI and 1836 rows for S2. And next we will use these metadata to co-locate them and produce another table shows the details of the colocation pairs.

## Co-locate the data

In this section we use the metadata we have just produced to produce the co-location pair details. The logic of the code is match rows from S2 and S3 OLCI by their geo_footprint.

In [ ]:
s3_olci_metadata = pd.read_csv(
    path_to_save_data + "sentinel3_olci_metadata.csv"
)
s2_metadata = pd.read_csv(
    path_to_save_data + "sentinel2_metadata.csv"
)

In [ ]:
s3_olci_metadata["ContentDate.Start"] = pd.to_datetime(
    s3_olci_metadata["ContentDate"].apply(lambda x: eval(x)["Start"])
).apply(make_timezone_naive)
s3_olci_metadata["ContentDate.End"] = pd.to_datetime(
    s3_olci_metadata["ContentDate"].apply(lambda x: eval(x)["End"])
).apply(make_timezone_naive)

s2_metadata["ContentDate.Start"] = pd.to_datetime(
    s2_metadata["ContentDate"].apply(lambda x: eval(x)["Start"])
).apply(make_timezone_naive)
s2_metadata["ContentDate.End"] = pd.to_datetime(
    s2_metadata["ContentDate"].apply(lambda x: eval(x)["End"])
).apply(make_timezone_naive)

results = check_collocation(
    s2_metadata, s3_olci_metadata, start_date, end_date,time_window=pd.to_timedelta("10 minutes")
)


As usual, you can have a look at the co-location output

In [ ]:
results.shape

(714, 8)

In [ ]:
from IPython.display import display

display(results.head(5))


,Satellite1_Name,Satellite1_ID,Satellite1_Footprint,Satellite2_Name,Satellite2_ID,Satellite2_Footprint,Overlap_Start,Overlap_End
0,S2A_MSIL1C_20180601T201851_N0500_R071_T16XEP_2...,e6e03e11-853c-49cb-9558-aa34b3c26f9c,"{'type': 'Polygon', 'coordinates': [[[-81.7193...",S3A_OL_1_EFR____20180601T201417_20180601T20171...,58c1eed1-098c-5071-8fa3-568f5b090a53,"{'type': 'Polygon', 'coordinates': [[[-156.343...",2018-06-01T20:14:17.050000,2018-06-01T20:17:17.050000
1,S2A_MSIL1C_20180601T201851_N0500_R071_T16XEP_2...,e6e03e11-853c-49cb-9558-aa34b3c26f9c,"{'type': 'Polygon', 'coordinates': [[[-81.7193...",S3B_OL_1_EFR____20180601T201324_20180601T20162...,1e6552b0-0ca1-5214-b28a-8bb9f64ad19b,"{'type': 'Polygon', 'coordinates': [[[-156.033...",2018-06-01T20:13:24.146000,2018-06-01T20:16:24.146000
2,S2A_MSIL2A_20180601T201851_N0500_R071_T16XEN_2...,7d345969-2abf-4de6-a9f4-3320d76c779b,"{'type': 'Polygon', 'coordinates': [[[-81.9366...",S3A_OL_1_EFR____20180601T201417_20180601T20171...,58c1eed1-098c-5071-8fa3-568f5b090a53,"{'type': 'Polygon', 'coordinates': [[[-156.343...",2018-06-01T20:14:17.050000,2018-06-01T20:17:17.050000
3,S2A_MSIL2A_20180601T201851_N0500_R071_T16XEN_2...,7d345969-2abf-4de6-a9f4-3320d76c779b,"{'type': 'Polygon', 'coordinates': [[[-81.9366...",S3B_OL_1_EFR____20180601T201324_20180601T20162...,1e6552b0-0ca1-5214-b28a-8bb9f64ad19b,"{'type': 'Polygon', 'coordinates': [[[-156.033...",2018-06-01T20:13:24.146000,2018-06-01T20:16:24.146000
4,S2A_MSIL1C_20180601T083651_N0500_R064_T41XNG_2...,9afdca10-4db6-49d5-b7f6-dab854f79873,"{'type': 'Polygon', 'coordinates': [[[62.99916...",S3A_OL_1_EFR____20180601T083023_20180601T08332...,1aeb345c-281d-5464-a22c-e4d97df8e5c5,"{'type': 'Polygon', 'coordinates': [[[19.8749,...",2018-06-01T08:30:22.504503,2018-06-01T08:33:22.504503


With code below, you can visualise the co-located footprint.

In [ ]:
from IPython.display import display

map_result = plot_results(results.head(50)) #for first five images
display(map_result)

### Proceeding with Sentinel-3 OLCI Download

Moving forward, we turn our attention to downloading the Sentinel-3 OLCI data. The process mirrors the approach we took with Sentinel-2, maintaining consistency in our methodology. We'll apply the same logic of filename conversion and follow the structured steps to retrieve the data from the Copernicus dataspace.

In [ ]:
download_dir = "/content/drive/MyDrive/Week_4"  # Replace with your desired download directory
product_id = results['Satellite1_ID'][0] # Replace with your desired file id
file_name = results['Satellite1_Name'][0]# Replace with your desired filename
# Download the single product
# download_single_product(product_id, file_name, access_token, download_dir)

## Sentinel-3 SRAL

It is also possible to co-locate S2/S3 OLCI with S3 SRAL (altimetry data). The overall logic is the same, we just need to fetch the S3 SRAL metadata.

In [ ]:
sentinel3_sral_data = query_sentinel3_sral_arctic_data(
    start_date, end_date, access_token
)

sentinel3_sral_data.to_csv(
    path_to_save_data + "s3_sral_metadata.csv",
    index=False,
)

And now you do the co-locaton again for S3 SRAL with S2 for example.

In [ ]:
s3_sral_metadata = pd.read_csv(
    path_to_save_data + "s3_sral_metadata.csv"
)
s2_metadata = pd.read_csv(
    path_to_save_data + "sentinel2_metadata.csv"
)

In [ ]:
s3_sral_metadata["ContentDate.Start"] = pd.to_datetime(
    s3_sral_metadata["ContentDate"].apply(lambda x: eval(x)["Start"])
).apply(make_timezone_naive)
s3_sral_metadata["ContentDate.End"] = pd.to_datetime(
    s3_sral_metadata["ContentDate"].apply(lambda x: eval(x)["End"])
).apply(make_timezone_naive)

s2_metadata["ContentDate.Start"] = pd.to_datetime(
    s2_metadata["ContentDate"].apply(lambda x: eval(x)["Start"])
).apply(make_timezone_naive)
s2_metadata["ContentDate.End"] = pd.to_datetime(
    s2_metadata["ContentDate"].apply(lambda x: eval(x)["End"])
).apply(make_timezone_naive)

results = check_collocation(
    s2_metadata, s3_sral_metadata, start_date, end_date,time_window=pd.to_timedelta("10 minutes")
)


And now you can plot the co-location results again.

In [ ]:
from IPython.display import display

map_result = plot_results(results.head(5))
display(map_result)